In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import requests
from bs4 import BeautifulSoup as bs 

In [7]:
riverflow_df = pd.read_csv("/Users/mugisha/Desktop/clone/Drought_Pakistan/mainInflowrivers.csv")
riverflow_df[riverflow_df["Year"]==2023]

,Date,INDUS,KABUL,JEHLUM,CHENAB,Year
364,2023-01-01,16.9,8.6,6.3,5.4,2023
365,2023-01-02,16.5,8.5,6.3,5.4,2023
366,2023-01-03,16.7,8.9,5.1,5.4,2023
367,2023-01-04,16.5,8.7,6.3,5.4,2023
368,2023-01-05,16.3,8.6,5.7,6.7,2023
...,...,...,...,...,...,...
425,2023-03-03,14.8,8.7,18.0,8.8,2023
426,2023-03-04,17.6,8.8,18.7,7.2,2023
427,2023-03-05,16.8,9.1,20.0,7.9,2023
428,2023-03-06,21.4,8.1,18.9,8.8,2023


In [17]:

riverflow_df['Year']= [year.split("-")[0] for year in list(riverflow_df.Date)]
riverflow_df

,Date,INDUS,KABUL,JEHLUM,CHENAB,Year
0,2022-01-01,16.9,4.2,7.6,5.9,2022
1,2022-01-02,15.2,4.6,8.3,5.9,2022
2,2022-01-03,16.4,4.1,8.3,6.3,2022
3,2022-01-04,15.0,4.2,7.6,6.3,2022
4,2022-01-05,17.5,4.4,7.6,6.3,2022
...,...,...,...,...,...,...
425,2023-03-03,14.8,8.7,18.0,8.8,2023
426,2023-03-04,17.6,8.8,18.7,7.2,2023
427,2023-03-05,16.8,9.1,20.0,7.9,2023
428,2023-03-06,21.4,8.1,18.9,8.8,2023


In [40]:

def crawl_riverflow(): 
    wapdaUrl = "http://www.wapda.gov.pk/index.php/river-flow-data"
    page = requests.get(wapdaUrl)

    soup = bs(page.content, "html.parser")

    table =  soup.find("table")

    # Extract the data from the table
    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        data.append(row_data)


    # # print(riverflowList)
    columnsNames = ["Date","LEVEL (FEET) INDUS AT TARBELA","INFLOW INDUS AT TARBELA", "OUTFLOW INDUS AT TARBELA",
                    "INFLOW KABUL AT NOWSHERA", "LEVEL (FEET) JEHLUM AT MANGLA", "INFLOW JEHLUM AT MANGLA",
                    "OUTFLOW JEHLUM AT MANGLA", "INFLOW CHENAB AT MARALA", "CURRENT YEAR", "LAST YEAR", "AVG: Last 10-years" ]

    riverflow_df = pd.DataFrame(columns=columnsNames)


    for row in data[4:]:
        riverflow_df.loc[len(riverflow_df)] = row

    
    return riverflow_df

def main_rivers():
    riverflow_df = crawl_riverflow()
    inflow_df = riverflow_df[["Date","INFLOW INDUS AT TARBELA", "INFLOW KABUL AT NOWSHERA",
                                               "INFLOW JEHLUM AT MANGLA", "INFLOW CHENAB AT MARALA"]]
    
    split_index = inflow_df.index.get_loc((inflow_df['Date'] == '1-Jan').idxmax())
                                           
    # dataframe corresponding to the year of 2022
    rf_2022_df = inflow_df.loc[:split_index].copy()
    dates_2022= [s.replace('/', '-') if '/' in s else s for s in rf_2022_df.Date]
    new_rf_2022_df= rf_2022_df.iloc[:,1:]
    new_rf_2022_df["Date"]= dates_2022
    rf_2022_reversed= new_rf_2022_df[::-1]
    final_2022_df= rf_2022_reversed.set_index("Date")
    final_2022_df.index = [pd.to_datetime(date_str + '-2022', format='%d-%b-%Y').strftime('%Y-%m-%d') for date_str in final_2022_df.index]


    # dataframe corresponding to the year of 2023
    rf_2023_df = inflow_df.loc[:split_index].copy()
    dates_2023= [s.replace('/', '-') if '/' in s else s for s in rf_2023_df.Date]
    new_rf_2023_df= rf_2023_df.iloc[:,1:]
    new_rf_2023_df["Date"]= dates_2023
    rf_2023_reversed= new_rf_2023_df[::-1]
    final_2023_df= rf_2023_reversed.set_index("Date")
    final_2023_df.index= [pd.to_datetime(date_str + '-2023', format='%d-%b-%Y').strftime('%Y-%m-%d') for date_str in final_2023_df.index]

    # combined dataframe for 2022 and 2023
    Inflow_combined_df= pd.concat([final_2022_df, final_2023_df], axis=0)
    Inflow_combined_df.reset_index(inplace=True)
    columns_names= ["Date","INDUS", "KABUL", "JEHLUM", "CHENAB"]
    Inflow_combined_df.columns= columns_names

    Inflow_combined_df['Year']= [year.split("-")[0] for year in list(Inflow_combined_df.Date)]

    return Inflow_combined_df


In [41]:
df = main_rivers()

In [42]:
df

,Date,INDUS,KABUL,JEHLUM,CHENAB,Year
0,2022-01-01,16.9,8.6,6.3,5.4,2022
1,2022-01-02,16.5,8.5,6.3,5.4,2022
2,2022-01-03,16.7,8.9,5.1,5.4,2022
3,2022-01-04,16.5,8.7,6.3,5.4,2022
4,2022-01-05,16.3,8.6,5.7,6.7,2022
...,...,...,...,...,...,...
127,2023-03-03,14.8,8.7,18.0,8.8,2023
128,2023-03-04,17.6,8.8,18.7,7.2,2023
129,2023-03-05,16.8,9.1,20.0,7.9,2023
130,2023-03-06,21.4,8.1,18.9,8.8,2023


In [31]:
split_index = df.index.get_loc((df['Date'] == '1-Jan').idxmax())

df1 = df.loc[:split_index].copy()
df2 = df.loc[split_index + 1:].copy()

In [39]:
df2

,Date,INFLOW INDUS AT TARBELA,INFLOW KABUL AT NOWSHERA,INFLOW JEHLUM AT MANGLA,INFLOW CHENAB AT MARALA
66,30-Dec,17.4,8.2,6.9,6.3
67,29-Dec,17.0,8.3,6.9,12.4
68,28-Dec,17.2,8.1,6.3,7.0
69,27-Dec,17.4,8.1,6.3,5.6
70,26-Dec,17.4,8.7,6.3,4.8
...,...,...,...,...,...
425,5-Jan,17.5,4.4,7.6,6.3
426,4-Jan,15.0,4.2,7.6,6.3
427,3-Jan,16.4,4.1,8.3,6.3
428,2-Jan,15.2,4.6,8.3,5.9
